In [50]:
import selenium
from selenium import webdriver
import os
import json
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import re
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import pandas as pd
from joblib import Parallel, delayed
import threading
import time


#from webdriver_manager.chrome import ChromeDriverManager 
#driver=webdriver.Chrome(ChromeDriverManager().install()) 

In [25]:
def scrolling_func(wait,driver):
    SCROLL_PAUSE_TIME = 1.0

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '.p-button')))
        load_button = driver.find_element_by_css_selector('.p-button')
        load_button.click()
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [26]:
def parse_dates(years,months):
    base_url = "https://www.theverge.com/archives" 
    urls = []
    zipped = zip(years,months)
    for year,month in zipped:
        url =  base_url+"/"+year+"/"+month
        urls.append(url)
    return urls

In [69]:
# class Scraper(threading.Thread):
#     def __init__(self, threadId, name, url,soup):
#         threading.Thread.__init__(self)
#         self.name = name
#         self.id = threadId
#         self.url = url

#     def run(self):
#         titles = soup.find_all("h2",class_="c-entry-box--compact__title")
#         dates = soup.find_all("time",class_="c-byline__item")
#         return titles,dates

In [135]:
def scraper(years,months):
    CHROME_PATH = r"C:\Users\astar\Stock market tutorials\chromedriver_win64\chromedriver.exe"
    driver = webdriver.Chrome(CHROME_PATH)
    driver.maximize_window()
    
    urls = parse_dates(years,months)
    
    headlines = []
    dates = []
    links = []

    for url in urls:
        driver.get(url)
        done=True
     
        
        while done:
            try:
                wait = WebDriverWait(driver,100)
                scrolling_func(wait,driver)  
            except:
                done=False

        soup = BeautifulSoup(driver.page_source,'lxml')
        #https://stackoverflow.com/questions/5041008/how-to-find-elements-by-class
        #https://stackoverflow.com/questions/42732958/python-parallel-execution-with-selenium
        #https://stackoverflow.com/questions/44245451/how-to-scrape-multiple-html-page-in-parallel-with-beautifulsoup-in-python
        for title in soup.find_all("h2",class_="c-entry-box--compact__title"):
            headlines.append(title.text)
            s = str(title)[str(title).find("href")+5:]
            links.append(s[:s.find(">")])
        for date in soup.find_all("time",class_="c-byline__item"):
            dates.append(date.text.strip())
        
        time.sleep(5)

    print(len(headlines),len(dates))    

    assert len(headlines)==len(dates), f"Different lengths of headlines {len(headlines)} and date {len(dates)}"
    data = {"Headlines":headlines,"Dates":dates,"Links":links}
    df = pd.DataFrame(data)
    driver.quit()    
    return df 

In [ ]:
df = scraper(["2021","2021"],["2","3"])

In [134]:
import requests
import urllib
url = "https://www.theverge.com/archives/2021/3"
url = requests.get(url)
soup = BeautifulSoup(url.text,'lxml')
titles = []
for title in soup.find_all("h2",class_="c-entry-box--compact__title"):
    s = str(title)[str(title).find("href")+5:]
    print(s[:s.find(">")])
    break

"https://www.theverge.com/2021/3/15/22332624/bird-europe-investment-scooter-expansion-new-hires"
